In [210]:
import seaborn as sns
import pandas as pd
import numpy as np
import functools
import operator
import collections

In [211]:
# Permanently changes the pandas settings
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)

## Abkürzungen

* LS: Lohnsteuer
* GKV: Gesetzliche Krankenversicherung
* PKV: Private Krankenversicherung

## Parameter

In [212]:
# Termintechnische Daten
JAHR = 2024
EINTRITTSALTER = 23
ENDALTER = 100

# Personenbezigene Daten
LOHNSTEUER = 0.42
SOLIDARITAETSZUSCHLAG = 0
STEUERLAST = LOHNSTEUER + SOLIDARITAETSZUSCHLAG
NETTOEINKOMMEN = 3300 # p.M.
EINKOMMENSENTWICKLUNG = 0.03 # p.a.

# Rente
RENTENEINTRITTSALTER = 70
RENTENANSPRUCH = 2500 # im Startjahr p.M.
RENTENENTWICKLUNG = 0.02 # p.a.
RENTENZUSCHUSS_PKV = 0.081
RENTENZUSCHUSS_PKV_MAX_ANTEIL = 0.5 # maximaler Anteil, der von der Rentenversicherung bezuschusst wird. 2024 max. 50%

# GKV (2024)
# https://www.bundesgesundheitsministerium.de/beitraege
GKV_BEITRAGSBEMESSUNGSGRENZE = 5175 # p.M.
GKV_BEITRAGSBEMESSUNGSGRENZE_STEIGERUNG = 0.02 # p.a.
GKV_BEITRAG = 0.146 # Anteil vom Bruttogehalt
GKV_ZUSATZBEITRAG = 0.023 # Anteil vom Bruttogehalt
GKV_ANTEIL_ARBEITGEBER = 0.5
GKV_ANTEIL_RENTENVERSICHERUNG = 0.5
GKV_GESAMTBEITRAG = GKV_BEITRAG + GKV_ZUSATZBEITRAG
GKV_GESAMTBEITRAG_INKL_AG = GKV_GESAMTBEITRAG * (1 - GKV_ANTEIL_ARBEITGEBER)
GKV_GESAMTBEITRAG_INKL_RV = GKV_GESAMTBEITRAG * (1 - GKV_ANTEIL_RENTENVERSICHERUNG)

# Pflegeversicherung in GKV
# https://www.tk.de/firmenkunden/versicherung/beitraege-faq/pflegereform-2023/wie-hoch-ist-pv-beitrag-ab-01072023-2149454?tkcm=ab
GKV_PV_BEITRAG = 0.034 # Anteil vom Bruttogehalt
GKV_PV_KINDERLOSENZUSCHLAG = 0.006 # Anteil vom Bruttogehalt / kein AG Zuschuss
GKV_PV_ANTEIL_ARBEITGEBER = 0.5
GKV_PV_KINDER_ABSCHLAG_BIS = 24 # "unter 25 Jahre"
GKV_PV_KINDER_ABSCHLAG_STAFFELUNG = {
    2: 0.0025, 
    3: 0.005, 
    4: 0.0075, 
    5: 0.01,
    6: 0.01,
    7: 0.01,
    8: 0.01,
    9: 0.01,
    10: 0.01,
}

# Zusatzversicherungen in GKV
GKV_ZV_BEITRAG = 0
GKV_ZV_BEITRAGSSTEIGERUNG = 0.035 # p.a.

# PKV Tarif
PKV_VERSICHERUNG = 'Alte Oldenburger'
PKV_SELBSTBETEILIGUNG = {
    'max': 330,
    'anteil': 0.2,
}
PKV_TARIFKONDITIONEN = {
    'A 80/100': {
        'beitrag': 250.16,
        'steuerlich_ansetzbar': 0.97,
        'wegfall_renteneintritt': False,
    },
    'K 20': {
        'beitrag': 127.41,
        'steuerlich_ansetzbar': 0.6205,
        'wegfall_renteneintritt': False,
    },
    'K/S': {
        'beitrag': 131.33 - 127.41,
        'steuerlich_ansetzbar': 0,
        'wegfall_renteneintritt': False,
    },
    'Z 100/80': {
        'beitrag': 58.95,
        'steuerlich_ansetzbar': 0.6285,
        'wegfall_renteneintritt': False,
    },
    'AK (Variante AK-E)': {
        'beitrag': 0.95,
        'steuerlich_ansetzbar': 0,
        'wegfall_renteneintritt': False,
    },
    'KTV 6 140€': {
        'beitrag': 40.04,
        'steuerlich_ansetzbar': 0,
        'wegfall_renteneintritt': True,
    },
}
PKV_BEITRAGSSTEIGERUNG = 0.035 # p.a.
PKV_ANTEIL_ARBEITGEBER = 0.5
PKV_BEITRAGSENTLASTUNGSPAKET = {
    'bezeichnung': 'PBE 2.0 350€',
    'beitrag': 141.40,
    'steuerlich_ansetzbar': -1, # wenn auf -1 gesetz wird der Anteil automatisch berechnet (durchschnittliche Absetzbarkeit)
}
PKV_BEITRAGSENTLASTUNGSPAKET_WIRKSAM_AB = 65 # Beitragssenkung ab 65
PKV_GESETZLICHE_BEITRAGSENTLASTUNG_EINZAHLUNG_BIS = 60 # Einzahlung bis x Jahre
PKV_GESETZLICHE_BEITRAGSENTLASTUNG_ANTEIL = 0.1
PKV_PV = {
    'bezeichnung': 'PVN',
    'beitrag': 50.47,
    'steuerlich_ansetzbar': 1,
}
PKV_PV_ANTEIL_ARBEITGEBER = 0.5

# Kinder
KINDER = [
    {
        'alter_bei_geburt': 30,
        'versicherungsdauer': 24, # in Jahren (bis zum 24ten Geburtstag)
    },
    {
        'alter_bei_geburt': 32,
        'versicherungsdauer': 24, # in Jahren (bis zum 24ten Geburtstag)
    },
]
PKV_KINDER_BEITRAG = 150 # in 2024 (Inflation wird im Rechner beachtet)
PKV_KINDER_BEITRAGSSTEIGERUNG = 0.035 # p.a.

# Kapitalmarkt und Wirtschaft
INFLATIONSRATE = 0.025 # p.a.
RENDITE_KAPITALMARKT = 0.06 # p.a.
KAPITALERTRAGSSTEUER = 0.25

## Durchschnittliche steuerliche Ansetzbarkeit

In [213]:
beitrag, _, _ = functools.reduce(operator.add, map(collections.Counter, PKV_TARIFKONDITIONEN.values())).values()
beitrag += PKV_PV['beitrag']
durchschnittliche_ansetzbarkeit = (PKV_PV['beitrag'] / beitrag) * PKV_PV['steuerlich_ansetzbar']
for konditionen in PKV_TARIFKONDITIONEN.values():
    durchschnittliche_ansetzbarkeit += (konditionen['beitrag'] / beitrag) * konditionen['steuerlich_ansetzbar']
if PKV_BEITRAGSENTLASTUNGSPAKET['steuerlich_ansetzbar'] == -1:
    PKV_BEITRAGSENTLASTUNGSPAKET['steuerlich_ansetzbar'] = durchschnittliche_ansetzbarkeit
durchschnittliche_ansetzbarkeit

0.7693799210377892

## Einkommensentwicklung

In [214]:
df = pd.DataFrame()

df['Jahr'] = np.arange(JAHR, JAHR - EINTRITTSALTER + ENDALTER + 1)
df['Alter'] = np.arange(EINTRITTSALTER, ENDALTER + 1)
df[f'Inflation seit {JAHR}'] = np.power(1 + INFLATIONSRATE, df['Jahr'] - JAHR)
df['Rentenbezug'] = df['Alter'] >= RENTENEINTRITTSALTER
df['Einkommen'] = NETTOEINKOMMEN * np.power(1 + EINKOMMENSENTWICKLUNG, df['Alter'] - EINTRITTSALTER) * (1 - df['Rentenbezug'])
df['Rente'] = RENTENANSPRUCH * np.power(1 + RENTENENTWICKLUNG, df['Alter'] - EINTRITTSALTER) * df['Rentenbezug']

df

,Jahr,Alter,Inflation seit 2024,Rentenbezug,Einkommen,Rente
0,2024,23,1.000000,False,3300.000000,0.000000
1,2025,24,1.025000,False,3399.000000,0.000000
2,2026,25,1.050625,False,3500.970000,0.000000
3,2027,26,1.076891,False,3605.999100,0.000000
4,2028,27,1.103813,False,3714.179073,0.000000
5,2029,28,1.131408,False,3825.604445,0.000000
6,2030,29,1.159693,False,3940.372579,0.000000
7,2031,30,1.188686,False,4058.583756,0.000000
8,2032,31,1.218403,False,4180.341269,0.000000
9,2033,32,1.248863,False,4305.751507,0.000000


## GKV Beiträge

In [215]:
kinder_sortiert = sorted(KINDER, key=lambda k: k['alter_bei_geburt'])
alter_wenn_erstes_kind = kinder_sortiert[0]['alter_bei_geburt'] if len(kinder_sortiert) > 0 else 1000

anzahl_kinder_pv_relevant = np.zeros(ENDALTER - EINTRITTSALTER + 1)
for kind in KINDER:
    start = kind['alter_bei_geburt'] - EINTRITTSALTER
    ende = start + GKV_PV_KINDER_ABSCHLAG_BIS
    for i in range(start, ende + 1):
        anzahl_kinder_pv_relevant[i] += 1


df['GKV: Kinderlos'] = df['Alter'] < alter_wenn_erstes_kind
df[f'GKV: Kinder bis {GKV_PV_KINDER_ABSCHLAG_BIS} Jahre'] = anzahl_kinder_pv_relevant.astype(int)
df[f'GKV: Kinder bis {GKV_PV_KINDER_ABSCHLAG_BIS} Jahre PV Abschlag'] = df[f'GKV: Kinder bis {GKV_PV_KINDER_ABSCHLAG_BIS} Jahre'].map(GKV_PV_KINDER_ABSCHLAG_STAFFELUNG).fillna(0)

df['GKV: Beitragsbemessungsgrenze'] = GKV_BEITRAGSBEMESSUNGSGRENZE * np.power(1 + GKV_BEITRAGSBEMESSUNGSGRENZE_STEIGERUNG, df['Alter'] - EINTRITTSALTER)

df['GKV: Beitrag inkl. AG Anteil'] = df['GKV: Beitragsbemessungsgrenze'] * GKV_GESAMTBEITRAG_INKL_AG * (1 - df['Rentenbezug'])
df['GKV: PV Beitrag inkl. AG Anteil'] = df['GKV: Beitragsbemessungsgrenze'] * (GKV_PV_BEITRAG * (1 - GKV_PV_ANTEIL_ARBEITGEBER) + GKV_PV_KINDERLOSENZUSCHLAG * df['GKV: Kinderlos'] - df[f'GKV: Kinder bis {GKV_PV_KINDER_ABSCHLAG_BIS} Jahre PV Abschlag']) * (1 - df['Rentenbezug'])
df['GKV: Gesamt Beitrag inkl. AG Anteil'] = df['GKV: Beitrag inkl. AG Anteil'] + df['GKV: PV Beitrag inkl. AG Anteil']

df['GKV: Beitrag Rente inkl. RV Anteil'] = np.minimum(df['Rente'], df['GKV: Beitragsbemessungsgrenze']) * GKV_GESAMTBEITRAG_INKL_RV * df['Rentenbezug']
df['GKV: PV Beitrag Rente'] = np.minimum(df['Rente'], df['GKV: Beitragsbemessungsgrenze']) * (GKV_PV_BEITRAG + GKV_PV_KINDERLOSENZUSCHLAG * df['GKV: Kinderlos'] - df[f'GKV: Kinder bis {GKV_PV_KINDER_ABSCHLAG_BIS} Jahre PV Abschlag']) * df['Rentenbezug']
df['GKV: Gesamt Beitrag Rente inkl. RV Anteil'] = df['GKV: Beitrag Rente inkl. RV Anteil'] + df['GKV: PV Beitrag Rente']

df['GKV: Beitrag inkl. Zuschüsse exkl. LS'] = np.where(df['Rentenbezug'], df['GKV: Gesamt Beitrag Rente inkl. RV Anteil'], df['GKV: Gesamt Beitrag inkl. AG Anteil'])
df['GKV: Steuererstattung'] = df['GKV: Beitrag inkl. Zuschüsse exkl. LS'] * STEUERLAST
df['GKV: Beitrag inkl. Zuschüsse inkl. LS'] = df['GKV: Beitrag inkl. Zuschüsse exkl. LS'] - df['GKV: Steuererstattung']

df

,Jahr,Alter,Inflation seit 2024,Rentenbezug,Einkommen,Rente,GKV: Kinderlos,GKV: Kinder bis 24 Jahre,GKV: Kinder bis 24 Jahre PV Abschlag,GKV: Beitragsbemessungsgrenze,GKV: Beitrag inkl. AG Anteil,GKV: PV Beitrag inkl. AG Anteil,GKV: Gesamt Beitrag inkl. AG Anteil,GKV: Beitrag Rente inkl. RV Anteil,GKV: PV Beitrag Rente,GKV: Gesamt Beitrag Rente inkl. RV Anteil,GKV: Beitrag inkl. Zuschüsse exkl. LS,GKV: Steuererstattung,GKV: Beitrag inkl. Zuschüsse inkl. LS
0,2024,23,1.000000,False,3300.000000,0.000000,True,0,0.0000,5175.000000,437.287500,119.025000,556.312500,0.000000,0.000000,0.000000,556.312500,233.651250,322.661250
1,2025,24,1.025000,False,3399.000000,0.000000,True,0,0.0000,5278.500000,446.033250,121.405500,567.438750,0.000000,0.000000,0.000000,567.438750,238.324275,329.114475
2,2026,25,1.050625,False,3500.970000,0.000000,True,0,0.0000,5384.070000,454.953915,123.833610,578.787525,0.000000,0.000000,0.000000,578.787525,243.090760,335.696764
3,2027,26,1.076891,False,3605.999100,0.000000,True,0,0.0000,5491.751400,464.052993,126.310282,590.363276,0.000000,0.000000,0.000000,590.363276,247.952576,342.410700
4,2028,27,1.103813,False,3714.179073,0.000000,True,0,0.0000,5601.586428,473.334053,128.836488,602.170541,0.000000,0.000000,0.000000,602.170541,252.911627,349.258914
5,2029,28,1.131408,False,3825.604445,0.000000,True,0,0.0000,5713.618157,482.800734,131.413218,614.213952,0.000000,0.000000,0.000000,614.213952,257.969860,356.244092
6,2030,29,1.159693,False,3940.372579,0.000000,True,0,0.0000,5827.890520,492.456749,134.041482,626.498231,0.000000,0.000000,0.000000,626.498231,263.129257,363.368974
7,2031,30,1.188686,False,4058.583756,0.000000,False,1,0.0000,5944.448330,502.305884,101.055622,603.361506,0.000000,0.000000,0.000000,603.361506,253.411832,349.949673
8,2032,31,1.218403,False,4180.341269,0.000000,False,1,0.0000,6063.337297,512.352002,103.076734,615.428736,0.000000,0.000000,0.000000,615.428736,258.480069,356.948667
9,2033,32,1.248863,False,4305.751507,0.000000,False,2,0.0025,6184.604043,522.599042,89.676759,612.275800,0.000000,0.000000,0.000000,612.275800,257.155836,355.119964


## PKV Beiträge

In [217]:
# Kinder
anzahl_mitversicherter_kinder = np.zeros(ENDALTER - EINTRITTSALTER + 1)
for kind in KINDER:
    start = kind['alter_bei_geburt'] - EINTRITTSALTER
    ende = start + kind['versicherungsdauer']
    for i in range(start, ende):
        anzahl_mitversicherter_kinder[i] += 1
df['PKV: Anzahl Kinder'] = anzahl_mitversicherter_kinder.astype(int)

# Beiträge vor Lohnsteuerbetrachtung
for tarif, konditionen in PKV_TARIFKONDITIONEN.items():
    df[f'PKV: Beitrag {tarif} exkl. LS'] = konditionen['beitrag'] * np.power(1 +  PKV_BEITRAGSSTEIGERUNG, df['Alter'] - EINTRITTSALTER) * np.where((df['Alter'] >= RENTENEINTRITTSALTER) & konditionen['wegfall_renteneintritt'], 0, 1)
df[f'PKV: Beitrag {PKV_BEITRAGSENTLASTUNGSPAKET["bezeichnung"]} exkl. LS'] = PKV_BEITRAGSENTLASTUNGSPAKET['beitrag'] * np.power(1 +  PKV_BEITRAGSSTEIGERUNG, df['Alter'] - EINTRITTSALTER)
df[f'PKV: Beitrag {PKV_PV["bezeichnung"]} exkl. LS'] = PKV_PV['beitrag'] * np.power(1 + PKV_BEITRAGSSTEIGERUNG, df['Alter'] - EINTRITTSALTER)
df['PKV: Beitrag Kinder exkl. LS'] = PKV_KINDER_BEITRAG * np.power(1 + PKV_BEITRAGSSTEIGERUNG, df['Alter'] - EINTRITTSALTER) * df['PKV: Anzahl Kinder']
df['PKV: Beitrag Gesamt exkl. PV exkl. LS'] = df.loc[:, df.columns.str.startswith('PKV: Beitrag ') & df.columns.str.endswith(' exkl. LS') & ~df.columns.str.contains('Gesamt') & ~df.columns.str.contains(PKV_PV["bezeichnung"]) & ~df.columns.str.contains("Zuschüsse")].sum(axis=1)

# gesetzliche Altersrückstellungen
df['PKV: gesetzlicher Zuschlag zur Beitragsentlastung'] = df['Alter'] < PKV_GESETZLICHE_BEITRAGSENTLASTUNG_EINZAHLUNG_BIS
anteil_gesetzliche_altersrueckstellungen = 1 - (1 / (1 + PKV_GESETZLICHE_BEITRAGSENTLASTUNG_ANTEIL))
df['PKV: Einzahlung gesetzliche Altersrückstellung'] = np.where(df['Alter'] <= PKV_GESETZLICHE_BEITRAGSENTLASTUNG_EINZAHLUNG_BIS, (df['PKV: Beitrag Gesamt exkl. PV exkl. LS'] - df['PKV: Beitrag Kinder exkl. LS']) * anteil_gesetzliche_altersrueckstellungen, 0)

# Beiträge mit entfall der gesetzlichen Altersrückstellungen Lohnsteuerbetrachtung
for tarif, konditionen in PKV_TARIFKONDITIONEN.items():
    df[f'PKV: Beitrag {tarif} exkl. LS'] *= np.where(df['PKV: gesetzlicher Zuschlag zur Beitragsentlastung'], 1, 1 - anteil_gesetzliche_altersrueckstellungen)
df[f'PKV: Beitrag {PKV_BEITRAGSENTLASTUNGSPAKET["bezeichnung"]} exkl. LS'] *= np.where(df['PKV: gesetzlicher Zuschlag zur Beitragsentlastung'], 1, 1 - anteil_gesetzliche_altersrueckstellungen)
df[f'PKV: Beitrag {PKV_PV["bezeichnung"]} exkl. LS'] *= np.where(df['PKV: gesetzlicher Zuschlag zur Beitragsentlastung'], 1, 1 - anteil_gesetzliche_altersrueckstellungen)
df['PKV: Beitrag Gesamt exkl. PV exkl. LS'] = df.loc[:, df.columns.str.startswith('PKV: Beitrag ') & df.columns.str.endswith(' exkl. LS') & ~df.columns.str.contains('Gesamt') & ~df.columns.str.contains(PKV_PV["bezeichnung"]) & ~df.columns.str.contains("Zuschüsse")].sum(axis=1)

# Zuschüsse AG
df['PKV: AG Zuschuss Gesamt exkl. PV'] = np.minimum(df['GKV: Beitragsbemessungsgrenze'] * GKV_GESAMTBEITRAG, df['PKV: Beitrag Gesamt exkl. PV exkl. LS']) * (1 - PKV_ANTEIL_ARBEITGEBER) * (1 - df['Rentenbezug'])
for tarif, konditionen in PKV_TARIFKONDITIONEN.items():
    df[f'PKV: AG Zuschuss {tarif}'] = (df[f'PKV: Beitrag {tarif} exkl. LS'] / df['PKV: Beitrag Gesamt exkl. PV exkl. LS']) * df['PKV: AG Zuschuss Gesamt exkl. PV'] * (1 - df['Rentenbezug'])
df['PKV: AG Zuschuss Kinder'] = (df['PKV: Beitrag Kinder exkl. LS'] / df['PKV: Beitrag Gesamt exkl. PV exkl. LS']) * df['PKV: AG Zuschuss Gesamt exkl. PV'] * (1 - df['Rentenbezug'])
df[f'PKV: AG Zuschuss {PKV_PV["bezeichnung"]}'] = np.minimum(df['GKV: Beitragsbemessungsgrenze'] * GKV_PV_BEITRAG, df[f'PKV: Beitrag {PKV_PV["bezeichnung"]} exkl. LS']) * (1 - PKV_PV_ANTEIL_ARBEITGEBER) * (1 - df['Rentenbezug'])
df[f'PKV: AG Zuschuss {PKV_BEITRAGSENTLASTUNGSPAKET["bezeichnung"]}'] = (df[f'PKV: Beitrag {PKV_BEITRAGSENTLASTUNGSPAKET["bezeichnung"]} exkl. LS'] / df['PKV: Beitrag Gesamt exkl. PV exkl. LS']) * df['PKV: AG Zuschuss Gesamt exkl. PV'] * (1 - df['Rentenbezug'])
df['PKV: AG Zuschuss Gesamt exkl. PV'] = df.loc[:, df.columns.str.startswith('PKV: AG Zuschuss ') & ~df.columns.str.contains('Gesamt') & ~df.columns.str.contains(PKV_PV["bezeichnung"])].sum(axis=1)

# Zuschüsse RV
df['PKV: RV Zuschuss Gesamt exkl. PV'] = np.minimum(df['Rente'] * RENTENZUSCHUSS_PKV, df['PKV: Beitrag Gesamt exkl. PV exkl. LS'] * RENTENZUSCHUSS_PKV_MAX_ANTEIL)
for tarif, konditionen in PKV_TARIFKONDITIONEN.items():
    df[f'PKV: RV Zuschuss {tarif}'] = (df[f'PKV: Beitrag {tarif} exkl. LS'] / df['PKV: Beitrag Gesamt exkl. PV exkl. LS']) * df['PKV: RV Zuschuss Gesamt exkl. PV'] * df['Rentenbezug']
df['PKV: RV Zuschuss Kinder'] = (df['PKV: Beitrag Kinder exkl. LS'] / df['PKV: Beitrag Gesamt exkl. PV exkl. LS']) * df['PKV: AG Zuschuss Gesamt exkl. PV'] * df['Rentenbezug']
df[f'PKV: RV Zuschuss {PKV_PV["bezeichnung"]}'] = 0
df[f'PKV: RV Zuschuss {PKV_BEITRAGSENTLASTUNGSPAKET["bezeichnung"]}'] = (df[f'PKV: Beitrag {PKV_BEITRAGSENTLASTUNGSPAKET["bezeichnung"]} exkl. LS'] / df['PKV: Beitrag Gesamt exkl. PV exkl. LS']) * df['PKV: RV Zuschuss Gesamt exkl. PV'] * df['Rentenbezug']
df['PKV: RV Zuschuss Gesamt exkl. PV'] = df.loc[:, df.columns.str.startswith('PKV: RV Zuschuss ') & ~df.columns.str.contains('Gesamt') & ~df.columns.str.contains(PKV_PV["bezeichnung"])].sum(axis=1)

# Beiträge inkl. Zuschüsse
for tarif, konditionen in PKV_TARIFKONDITIONEN.items():
    df[f'PKV: Beitrag {tarif} inkl. Zuschüsse exkl. LS'] = df[f'PKV: Beitrag {tarif} exkl. LS'] - df[f'PKV: AG Zuschuss {tarif}'] - df[f'PKV: RV Zuschuss {tarif}']
df['PKV: Beitrag Kinder inkl. Zuschüsse exkl. LS'] = df['PKV: Beitrag Kinder exkl. LS'] - df['PKV: AG Zuschuss Kinder'] - df['PKV: RV Zuschuss Kinder']
df[f'PKV: Beitrag {PKV_PV["bezeichnung"]} inkl. Zuschüsse exkl. LS'] = df[f'PKV: Beitrag {PKV_PV["bezeichnung"]} exkl. LS'] - df[f'PKV: AG Zuschuss {PKV_PV["bezeichnung"]}'] - df[f'PKV: RV Zuschuss {PKV_PV["bezeichnung"]}']
df[f'PKV: Beitrag {PKV_BEITRAGSENTLASTUNGSPAKET["bezeichnung"]} inkl. Zuschüsse exkl. LS'] = df[f'PKV: Beitrag {PKV_BEITRAGSENTLASTUNGSPAKET["bezeichnung"]} exkl. LS'] - df[f'PKV: AG Zuschuss {PKV_BEITRAGSENTLASTUNGSPAKET["bezeichnung"]}'] - df[f'PKV: RV Zuschuss {PKV_BEITRAGSENTLASTUNGSPAKET["bezeichnung"]}']
df['PKV: Beitrag Gesamt inkl. Zuschüsse exkl. PV exkl. LS'] = df.loc[:, df.columns.str.startswith('PKV: Beitrag ') & df.columns.str.endswith(' inkl. Zuschüsse exkl. LS') & ~df.columns.str.contains('Gesamt') & ~df.columns.str.contains(PKV_PV["bezeichnung"])].sum(axis=1)

# Steuererstattung
for tarif, konditionen in PKV_TARIFKONDITIONEN.items():
    df[f'PKV: Steuererstattung {tarif}'] = df[f'PKV: Beitrag {tarif} inkl. Zuschüsse exkl. LS'] * konditionen['steuerlich_ansetzbar'] * STEUERLAST
df['PKV: Steuererstattung Kinder'] = df['PKV: Beitrag Kinder inkl. Zuschüsse exkl. LS'] * STEUERLAST
df[f'PKV: Steuererstattung {PKV_PV["bezeichnung"]}'] = df[f'PKV: Beitrag {PKV_PV["bezeichnung"]} inkl. Zuschüsse exkl. LS'] * PKV_PV['steuerlich_ansetzbar'] * STEUERLAST
df[f'PKV: Steuererstattung {PKV_BEITRAGSENTLASTUNGSPAKET["bezeichnung"]}'] = df[f'PKV: Beitrag {PKV_BEITRAGSENTLASTUNGSPAKET["bezeichnung"]} inkl. Zuschüsse exkl. LS'] * PKV_BEITRAGSENTLASTUNGSPAKET['steuerlich_ansetzbar'] * STEUERLAST
df['PKV: Steuererstattung Gesamt'] = df.loc[:, df.columns.str.startswith('PKV: Steuererstattung ') & ~df.columns.str.contains('Gesamt')].sum(axis=1)

# Beiträge nach Steuern
for tarif, konditionen in PKV_TARIFKONDITIONEN.items():
    df[f'PKV: Beitrag {tarif} inkl. Zuschüsse inkl. LS'] = df[f'PKV: Beitrag {tarif} inkl. Zuschüsse exkl. LS'] - df[f'PKV: Steuererstattung {tarif}']
df['PKV: Beitrag Kinder inkl. Zuschüsse inkl. LS'] = df['PKV: Beitrag Kinder inkl. Zuschüsse exkl. LS'] - df['PKV: Steuererstattung Kinder']
df[f'PKV: Beitrag {PKV_PV["bezeichnung"]} inkl. Zuschüsse inkl. LS'] = df[f'PKV: Beitrag {PKV_PV["bezeichnung"]} inkl. Zuschüsse exkl. LS'] - df[f'PKV: Steuererstattung {PKV_PV["bezeichnung"]}']
df[f'PKV: Beitrag {PKV_BEITRAGSENTLASTUNGSPAKET["bezeichnung"]} inkl. Zuschüsse inkl. LS'] = df[f'PKV: Beitrag {PKV_BEITRAGSENTLASTUNGSPAKET["bezeichnung"]} inkl. Zuschüsse exkl. LS'] - df[f'PKV: Steuererstattung {PKV_BEITRAGSENTLASTUNGSPAKET["bezeichnung"]}']
df['PKV: Beitrag Gesamt inkl. Zuschüsse inkl. LS'] = df.loc[:, df.columns.str.startswith('PKV: Beitrag ') & df.columns.str.endswith(' inkl. Zuschüsse inkl. LS') & ~df.columns.str.contains('Gesamt')].sum(axis=1)

df.loc[:, ~df.columns.str.startswith('GKV: ')]

,Jahr,Alter,Inflation seit 2024,Rentenbezug,Einkommen,Rente,PKV: Anzahl Kinder,PKV: Beitrag A 80/100 exkl. LS,PKV: Beitrag K 20 exkl. LS,PKV: Beitrag K/S exkl. LS,PKV: Beitrag Z 100/80 exkl. LS,PKV: Beitrag AK (Variante AK-E) exkl. LS,PKV: Beitrag KTV 6 140€ exkl. LS,PKV: Beitrag PBE 2.0 350€ exkl. LS,PKV: Beitrag PVN exkl. LS,PKV: Beitrag Kinder exkl. LS,PKV: Beitrag Gesamt exkl. PV exkl. LS,PKV: gesetzlicher Zuschlag zur Beitragsentlastung,PKV: Einzahlung gesetzliche Altersrückstellung,PKV: AG Zuschuss Gesamt exkl. PV,PKV: AG Zuschuss A 80/100,PKV: AG Zuschuss K 20,PKV: AG Zuschuss K/S,PKV: AG Zuschuss Z 100/80,PKV: AG Zuschuss AK (Variante AK-E),PKV: AG Zuschuss KTV 6 140€,PKV: AG Zuschuss Kinder,PKV: AG Zuschuss PVN,PKV: AG Zuschuss PBE 2.0 350€,PKV: RV Zuschuss Gesamt exkl. PV,PKV: RV Zuschuss A 80/100,PKV: RV Zuschuss K 20,PKV: RV Zuschuss K/S,PKV: RV Zuschuss Z 100/80,PKV: RV Zuschuss AK (Variante AK-E),PKV: RV Zuschuss KTV 6 140€,PKV: RV Zuschuss Kinder,PKV: RV Zuschuss PVN,PKV: RV Zuschuss PBE 2.0 350€,PKV: Beitrag A 80/100 inkl. Zuschüsse exkl. LS,PKV: Beitrag K 20 inkl. Zuschüsse exkl. LS,PKV: Beitrag K/S inkl. Zuschüsse exkl. LS,PKV: Beitrag Z 100/80 inkl. Zuschüsse exkl. LS,PKV: Beitrag AK (Variante AK-E) inkl. Zuschüsse exkl. LS,PKV: Beitrag KTV 6 140€ inkl. Zuschüsse exkl. LS,PKV: Beitrag Kinder inkl. Zuschüsse exkl. LS,PKV: Beitrag PVN inkl. Zuschüsse exkl. LS,PKV: Beitrag PBE 2.0 350€ inkl. Zuschüsse exkl. LS,PKV: Beitrag Gesamt inkl. Zuschüsse exkl. PV exkl. LS,PKV: Steuererstattung A 80/100,PKV: Steuererstattung K 20,PKV: Steuererstattung K/S,PKV: Steuererstattung Z 100/80,PKV: Steuererstattung AK (Variante AK-E),PKV: Steuererstattung KTV 6 140€,PKV: Steuererstattung Kinder,PKV: Steuererstattung PVN,PKV: Steuererstattung PBE 2.0 350€,PKV: Steuererstattung Gesamt,PKV: Beitrag A 80/100 inkl. Zuschüsse inkl. LS,PKV: Beitrag K 20 inkl. Zuschüsse inkl. LS,PKV: Beitrag K/S inkl. Zuschüsse inkl. LS,PKV: Beitrag Z 100/80 inkl. Zuschüsse inkl. LS,PKV: Beitrag AK (Variante AK-E) inkl. Zuschüsse inkl. LS,PKV: Beitrag KTV 6 140€ inkl. Zuschüsse inkl. LS,PKV: Beitrag Kinder inkl. Zuschüsse inkl. LS,PKV: Beitrag PVN inkl. Zuschüsse inkl. LS,PKV: Beitrag PBE 2.0 350€ inkl. Zuschüsse inkl. LS,PKV: Beitrag Gesamt inkl. Zuschüsse inkl. LS
0,2024,23,1.000000,False,3300.000000,0.000000,0,250.160000,127.410000,3.920000,58.950000,0.950000,40.040000,141.400000,50.470000,0.000000,622.830000,True,56.620909,311.415000,125.080000,63.705000,1.960000,29.475000,0.475000,20.020000,0.000000,25.235000,70.700000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0,0.000000,125.080000,63.705000,1.960000,29.475000,0.475000,20.020000,0.000000,25.235000,70.700000,311.415000,50.957592,16.602160,0.0,7.780516,0.0,0.0,0.000000,10.598700,22.845967,108.784935,74.122408,47.102840,1.960000,21.694484,0.475000,20.020000,0.000000,14.636300,47.854033,227.865065
1,2025,24,1.025000,False,3399.000000,0.000000,0,258.915600,131.869350,4.057200,61.013250,0.983250,41.441400,146.349000,52.236450,0.000000,644.629050,True,58.602641,322.314525,129.457800,65.934675,2.028600,30.506625,0.491625,20.720700,0.000000,26.118225,73.174500,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0,0.000000,129.457800,65.934675,2.028600,30.506625,0.491625,20.720700,0.000000,26.118225,73.174500,322.314525,52.741108,17.183236,0.0,8.052834,0.0,0.0,0.000000,10.969654,23.645576,112.592408,76.716692,48.751439,2.028600,22.453791,0.491625,20.720700,0.000000,15.148570,49.528924,235.840342
2,2026,25,1.050625,False,3500.970000,0.000000,0,267.977646,136.484777,4.199202,63.148714,1.017664,42.891849,151.471215,54.064726,0.000000,667.191067,True,60.653733,333.595533,133.988823,68.242389,2.099601,31.574357,0.508832,21.445925,0.000000,27.032363,75.735608,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0,0.000000,133.988823,68.242389,2.099601,31.574357,0.508832,21.445924,0.000000,27.032363,75.735608,333.595533,54.587046,17.784649,0.0,8.334683,0.0,0.0,0.000000,11.3535

## Vermögensaufbau